In [1]:
# Setup
from datetime import datetime
import os

import pandas as pd

from utils import render

transactions = pd.read_csv(os.path.join('data', 'transactions.csv'), index_col=0)
requests = pd.read_csv(os.path.join('data', 'requests.csv'), index_col=0)

# Pop out a quick sanity check
(transactions.shape, requests.shape)

((998, 4), (313, 4))

In [2]:
transactions.head(2)

,sender,receiver,amount,sent_date
0,stein,smoyer,49.03,2018-01-24
1,holden4580,joshua.henry,34.64,2018-02-06


In [3]:
requests.head(2)

,from_user,to_user,amount,request_date
0,chad.chen,paula7980,78.61,2018-02-12
1,kallen,lmoore,1.94,2018-02-23


In [4]:
#we are trying to look for all the requests that have a matching transaction
# Since we are calling merge on the `requests` DataFrame it is considered the left side
successful_requests = requests.merge(
    # And transactions is the right side
    transactions, 
    # So now we line up columns that will make the join make sense.
    left_on=['from_user', 'to_user', 'amount'], 
    right_on=['receiver', 'sender', 'amount']
)
# Let's take a look and see how we did
successful_requests.head()
#since amount is the same only one column is represented

,from_user,to_user,amount,request_date,sender,receiver,sent_date
0,chad.chen,paula7980,78.61,2018-02-12,paula7980,chad.chen,2018-07-15
1,kallen,lmoore,1.94,2018-02-23,lmoore,kallen,2018-03-05
2,gregory.blackwell,rodriguez5768,30.57,2018-03-04,rodriguez5768,gregory.blackwell,2018-03-17
3,kristina.miller,john.hardy,77.05,2018-03-12,john.hardy,kristina.miller,2018-04-25
4,lacey8987,mcguire,54.09,2018-03-13,mcguire,lacey8987,2018-06-28


In [5]:
# we now want to see time difference between the transation request and transacted date
successful_requests.dtypes

from_user        object
to_user          object
amount          float64
request_date     object
sender           object
receiver         object
sent_date        object
dtype: object

In [6]:
# changing date columns to date types
successful_requests['request_date'] = pd.to_datetime(successful_requests['request_date'])
successful_requests['sent_date'] = pd.to_datetime(successful_requests['sent_date'])
# And now we can see they are converted
successful_requests.dtypes

from_user               object
to_user                 object
amount                 float64
request_date    datetime64[ns]
sender                  object
receiver                object
sent_date       datetime64[ns]
dtype: object

In [7]:
# we can not subtract the dates, notice the vectorization
successful_requests['time_passed'] = successful_requests.sent_date - successful_requests.request_date

In [8]:
# look for top 5 longest transaction times
successful_requests.sort_values(by='time_passed', ascending=False).head(5)

,from_user,to_user,amount,request_date,sender,receiver,sent_date,time_passed
0,chad.chen,paula7980,78.61,2018-02-12,paula7980,chad.chen,2018-07-15,153 days
33,sthompson,andrade,14.07,2018-05-09,andrade,sthompson,2018-09-21,135 days
4,lacey8987,mcguire,54.09,2018-03-13,mcguire,lacey8987,2018-06-28,107 days
53,marcus.berry,melissa.mendoza,71.48,2018-05-31,melissa.mendoza,marcus.berry,2018-09-06,98 days
39,bishop,massey2102,18.27,2018-05-16,massey2102,bishop,2018-08-15,91 days


In [9]:
"Wow! ${:,.2f} has passed through the request system in {} transactions!!!".format(
    successful_requests.amount.sum(),
    len(successful_requests),
)

'Wow! $10,496.47 has passed through the request system in 214 transactions!!!'

In [10]:
#looking for duplicates 
# Create a boolean Series of records that are duplicated. 
#   Note that `keep=False` marks all that are duplicated
dupes = requests[requests.duplicated(('from_user', 'to_user', 'amount'), keep=False)]
# Order by requester and the date of request. 
#   Note that `request_date` in this case is a string, but this string date format sorts properly still.
dupes.sort_values(['from_user', 'request_date'])

,from_user,to_user,amount,request_date
58,austin486,shelly,11.24,2018-05-29
59,austin486,shelly,11.24,2018-05-29
8,cjimenez,sarah.evans,48.14,2018-03-21
26,cjimenez,sarah.evans,48.14,2018-04-27
218,clark8139,moore,14.54,2018-08-31
240,clark8139,moore,14.54,2018-09-10
195,diane4652,dean2365,6.82,2018-08-21
224,diane4652,dean2365,6.82,2018-09-05
143,donna1922,danderson,79.22,2018-07-23
157,donna1922,danderson,79.22,2018-07-31


## Handling Duplicated and Missing Data

In [11]:
# Setup
from datetime import datetime
import os

import numpy as np
import pandas as pd

from utils import render

users = pd.read_csv(os.path.join('data', 'users.csv'), index_col=0)
transactions = pd.read_csv(os.path.join('data', 'transactions.csv'), index_col=0)
requests = pd.read_csv(os.path.join('data', 'requests.csv'), index_col=0)

# Perform the merge from the previous notebook 
#  (s2n06-combining-dataframes.ipynb)
successful_requests = requests.merge(
    transactions,
    left_on=['from_user', 'to_user', 'amount'], 
    right_on=['receiver', 'sender', 'amount']
)

# Statement from previous notebook
"Wow! ${:,.2f} has passed through the request system in {} transactions!!!".format(
    successful_requests.amount.sum(),
    len(successful_requests),
)

'Wow! $10,496.47 has passed through the request system in 214 transactions!!!'

In [12]:
#we will now be trying to drop duplicate values
# Let's get our records sorted chronologically
successful_requests.sort_values('request_date', inplace=True) 

# And then we'll drop dupes keeping only the last one. 
#  Note the use of the inplace keyword
successful_requests.drop_duplicates(('from_user', 'to_user', 'amount'), keep='last', inplace=True)

# Statement from previous notebook
"Wow! ${:,.2f} has passed through the request system in {} transactions!!!".format(
    successful_requests.amount.sum(),
    len(successful_requests),
)
# always look for duplicates when merging dataframes!

'Wow! $9,316.12 has passed through the request system in 191 transactions!!!'

In [14]:
#looking for users who have made a request
# Create a boolean array where we check to see if the label (username) 
#   is in the `from_user` Series.
made_request_index = users.index.isin(requests.from_user)
# This will get us a list of all users who **have** made a request
users[made_request_index].head()

,first_name,last_name,email,email_verified,signup_date,referral_count,balance
aaron,Aaron,Davis,aaron6348@gmail.com,True,2018-08-31,6,18.14
acook,Anthony,Cook,cook@gmail.com,True,2018-05-12,2,55.45
adam.saunders,Adam,Saunders,adam@gmail.com,False,2018-05-29,3,72.12
adrian,Adrian,Fang,adrian.fang@teamtreehouse.com,True,2018-04-28,3,30.01
adrian.blair,Adrian,Blair,adrian9335@gmail.com,True,2018-06-16,7,25.85


In [15]:
#use ~ to get users who have not made a request
users[~made_request_index].head()

,first_name,last_name,email,email_verified,signup_date,referral_count,balance
alan9443,Alan,Pope,pope@hotmail.com,True,2018-04-17,0,56.09
alvarado,Denise,Alvarado,alvarado@hotmail.com,True,2018-09-07,6,26.72
amiller,Anne,Miller,miller@hotmail.com,False,2018-06-02,5,86.28
andersen,Mark,Andersen,mark.andersen@yahoo.com,True,2018-08-21,3,75.69
andrade,Melissa,Andrade,mandrade@yahoo.com,True,2018-01-06,3,83.22


In [16]:
# now we look for missing data
users.count()

first_name        475
last_name         430
email             475
email_verified    475
signup_date       475
referral_count    475
balance           475
dtype: int64

In [17]:
# Gather all users where the last name is missing
users[users.last_name.isna()].head()

,first_name,last_name,email,email_verified,signup_date,referral_count,balance
anthony3761,Anthony,NaN,anthony9581@gmail.com,True,2018-08-16,6,59.81
bradley,Bradley,NaN,bradley3941@henderson.com,True,2018-06-24,4,22.24
bradley7808,Bradley,NaN,bradley8794@yahoo.com,True,2018-06-15,1,76.49
brenda,Brenda,NaN,brenda@hotmail.com,True,2018-07-28,7,6.18
brooke2027,Brooke,NaN,brooke6938@gmail.com,False,2018-05-23,0,7.22


In [18]:
#fill unknown last name values
# Make a copy of the DataFrame with "Unknown" as the last name where it is missing
users_with_unknown = users.fillna('Unknown')

# Make sure we got 'em all
users_with_unknown[users_with_unknown.last_name.isna()]

,first_name,last_name,email,email_verified,signup_date,referral_count,balance


In [19]:
#or we can drop the unknown last names
users_with_last_names = users.dropna()

# Row counts of the original 
(len(users), len(users_with_last_names))

(475, 430)